In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
df = pd.read_csv(r"C:\Users\maill\Documents\GitHub\SARA\data\fusion_def_clean.csv", low_memory=False)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [4]:
df.shape

(2192779, 48)

In [5]:
df['grav'].unique()

array(['Blessé léger', 'Blessé hospitalisé', 'Indemne', 'Tué'],
      dtype=object)

In [6]:
from sklearn.preprocessing import LabelEncoder

# Création d'une instance de LabelEncoder
le = LabelEncoder()

# Ajustement de LabelEncoder et transformation de la variable cible
df['grav'] = le.fit_transform(df['grav'])

In [7]:
from sklearn.preprocessing import OneHotEncoder

# Créer la variable cible y avant l'encodage
y = df['grav']

# Supprimer 'grav' du dataframe principal
df = df.drop('grav', axis=1)

ohe = OneHotEncoder(sparse=False)

# Limite pour le passage de l'encodage One-Hot à l'encodage de fréquence
limite = 10

for col in df.columns:
    # Compter le nombre de valeurs uniques dans la colonne
    unique_values = df[col].nunique()
    
    if unique_values <= limite:
        # Encodage One-Hot pour les colonnes avec moins de n valeurs uniques
        encoded_col = ohe.fit_transform(df[[col]])
        
        # Ajout des colonnes encodées au dataframe original
        for (k, categories) in enumerate(ohe.categories_):
            for (i, category) in enumerate(categories):
                df[f'{col}_{category}'] = encoded_col[:, i+k]
        
        # Supprimer la colonne originale
        df.drop(col, axis=1, inplace=True)
        
    else:
        # Encodage de fréquence pour les colonnes avec plus de n valeurs uniques
        freq_encoder = df[col].value_counts(normalize=True)
        df[col] = df[col].map(freq_encoder)



In [8]:
# Définir les caractéristiques (X) et la cible (y)
X = df

# Diviser l'ensemble de données en un ensemble d'entraînement et un ensemble de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
from sklearn.ensemble import RandomForestClassifier

#Créer une instance du modèle
model = RandomForestClassifier(random_state=42)

#Entrainer le modèle
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [10]:
from sklearn.metrics import classification_report

# Prédire les valeurs pour l'ensemble de test
y_pred = model.predict(X_test)

# Evaluer le modèle
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.56      0.45      0.50     89903
           1       0.65      0.64      0.65    157066
           2       0.73      0.86      0.79    179888
           3       0.47      0.03      0.06     11699

    accuracy                           0.68    438556
   macro avg       0.61      0.50      0.50    438556
weighted avg       0.66      0.68      0.66    438556



Classe 0 (Blessé léger) : Le modèle a du mal à correctement identifier les individus de cette classe.

Classe 1 (Blessé hospitalisé) : Le modèle a une meilleure performance pour cette classe par rapport à la classe 0

Classe 2 (Indemne) : Le modèle performe le mieux pour cette classe, avec une précision de 0.73 et un rappel de 0.86, ce qui signifie qu'il est très bon pour identifier correctement les individus de cette classe. 

Classe 3 (Tué) : Malheureusement, le modèle a du mal à identifier cette classe avec une précision de 0.47 et un très faible rappel de 0.03. Il identifie de manière incorrecte un grand nombre d'individus de cette classe et manque également beaucoup d'individus qui sont réellement de cette classe.

Il semble que le modèle pourrait avoir besoin d'une certaine optimisation pour améliorer ses performances, en particulier pour les classes 0 et 3. 

Il faudra surement étudier d'autre type de modèle.

In [11]:
import pickle

with open('random_forest_model.pkl','wb') as file:
    pickle.dump(model, file)